In [1]:
from thesis_initialise import *

In [2]:
from collections.abc import Mapping as _Mapping
import numbers as _numbers
import functools as _functools
import inspect as _inspect
import weakref as _weakref

from everest.h5anchor import disk as _disk
from everest import reseed as _reseed
from everest.funcy import (
    Function as _Function,
    generic as _generic,
    special as _special,
    Fn as _Fn,
    )
from everest.cascade import (
    Cascade as _Cascade,
    Inputs as _Inputs,
    )
from everest.funcy.base import Base as _Base
from everest import wordhash as _wordhash
from everest.exceptions import *

In [3]:
class A:
    @classmethod
    def foo(cls):
        print(cls)
class B(A):
    @classmethod
    def foo(cls):
        super().foo()
        print(cls)

In [4]:
class A(type):
    def _shape(self):
        return ()
    @property
    def shape(self):
        return self._shape()
class B(metaclass = A):
    @classmethod
    def _shape(cls):
        return tuple((*type(cls)._shape(cls,), 1))
B.shape

# class A(type):
#     ...
# class B(A):
#     ...
# class C(metaclass = A):
#     ...
# class D(metaclass = B):
#     ...
# class E(C):
#     ...
# class F(D):
#     ...

(1,)

In [19]:
################################################################################

class Schema(_generic.FuncyIncisable, type):
    userdefined = False
    _premade = _weakref.WeakValueDictionary()
    def __new__(meta, name, bases, dic, *args, **kwargs):
        cls = super().__new__(meta, name, bases, dic)
        cls.root = cls
        if cls.userdefined:
            cls.script = script = \
                str(_reseed.digits(12)) # <- TEMPORARY
    #             _disk.ToOpen(inspect.getfile(schema))()
            cls._schemaID = schemaID = \
                _wordhash.get_random_proper(2, seed = script)
            try:
                cls = meta._premade[schemaID]
                assert cls.script == script, (script[:32], schema.script[:32])
                raise KeyError
            except KeyError:
                meta._premade[schemaID] = cls
        else:
            cls._schemaID = f'EverestSchema_{cls.__name__}'
            return cls
        return cls
    def __init__(self, *args, defaults = None, **kwargs):
        if not defaults is None:
            self._defaults = defaults
        self.cases = _weakref.WeakValueDictionary()
        self.oids = _weakref.WeakValueDictionary()
        super().__init__(*args, **kwargs)
    @property
    def defaults(self):
        try:
            return self._defaults
        except AttributeError:
            defaults = self._defaults = _Inputs(
                self.__init__,
                name = self.__name__
                )
            return defaults
    @property
    def schemaID(self):
        return self.root._schemaID
    @property
    def hashID(self):
        return self.schemaID
    @property
    def shape(self) -> tuple:
        return (_special.infint,)
    @property
    def sibling(self) -> type:
        return Oid
    @property
    def child(self) -> type:
        return Case
    def _getitem_strict(self, incisor):
        return self._get_case(**incisor)
    def _get_case(self, *args, **kwargs):
        case = _Cascade(*args, **kwargs)
        try:
            case = self.cases[(hashID := case.hashID)]
        except KeyError:
            case = self.child(
                f"{self.root.__name__}:{case.hashID}",
                (self, *self.__bases__),
                dict(self.__dict__),
#                 dict(),
                case = case,
                defaults = self.defaults,
                )
            case.root = self.root
            self.cases[hashID] = case
        return case
    def _getitem_broad(self, incisor):
        if not isinstance(incisor, _Function):
            incisor = _Fn[incisor]
        return self._get_oid(incisor)
    def _getitem_deep(*_, **__):
        raise NotYetImplemented
    def _getitem_sub(*_, **__):
        raise NotYetImplemented
    def _get_oid(self, incisor):
        try:
            oid = self.oids[(hashID := incisor.hashID)]
        except KeyError:
            oid = self.sibling(
                f"{self.root.__name__}[{repr(incisor)}]",
                (self, *self.__bases__),
                dict(self.__dict__),
#                 dict(),
                source = self.root, incisor = incisor,
                )
            oid.root = self.root
            self.oids[oid.hashID] = oid
        return oid
    def _getitem_sub(self, arg):
        raise ValueError("Can't go that deep!")
    @classmethod
    def _get_incision_method(cls, arg):
        if isinstance(arg, _Function):
            if arg.isSeq:
                return cls._getitem_broad
        return super()._get_incision_method(arg)
    def __call__(self, *args, **kwargs):
        return self._get_case(*args, **kwargs)(**kwargs)

class Oid(Schema, _generic.FuncyBroadIncision):
    def _getitem_strict(self, incisor):
        return self.root._getitem_strict(self.incisor.sub[incisor].value)
    def _getitem_broad(self, incisor):
        if not isinstance(incisor, _Function):
            incisor = _Fn[incisor]
        incisor = self.incisor.sub[incisor]
        return self._get_oid(incisor)
    @property
    def hashID(self):
        return self.schemaID + f"[{self.incisor.hashID}]"
    @property
    def shape(self) -> tuple:
        return (_special.infint,)
    def __call__(self, *args, **kwargs):
        raise Exception("Cannot call Oid.")

class Case(Schema):
    def __init__(self, *args, case: _Cascade, **kwargs):
        self._case = case
        self._inputs = self.defaults.copy(**case)
        super().__init__(*args, **kwargs)
    @property
    def inputs(self):
        return self._inputs
    @property
    def case(self):
        return self._case
    @property
    def hashID(self):
        return self.schemaID + ':' + self.case.hashID
    @property
    def shape(self):
        _, *dims = super().shape
        return tuple(dims)
    @property
    def sibling(self) -> type:
        raise NotYetImplemented
    @property
    def child(self) -> type:
        raise NotYetImplemented
    def __call__(self, **kwargs):
        instance = object.__new__(self)
        inputs = self.inputs
        setArgs, setKwargs = inputs.args, inputs.kwargs
        instance.__init__(*setArgs, **{**kwargs, **setKwargs})
        instance.case = self.case
        instanceID = str(_reseed.digits(12))
        instance.instanceID = instanceID
        instance.hashID = self.hashID + ';' + instanceID
        instance.inputs = inputs
        return instance

################################################################################

class Basic(metaclass = Schema):
    ...

################################################################################

class MyClass(Basic):
    userdefined = False
    def __init__(self,
            a = 1,
            b = 2,
            c = 3,
            _d = 4,
            **kwargs,
            ):
        self.foo = a * b * c
        super().__init__(**kwargs)

In [20]:
myobj = MyClass(a = 10)
print(repr(myobj))
print(myobj.hashID)
print(myobj.inputs)
print(myobj.foo)

<__main__.MyClass:insignificant-cattery object at 0x7f549fc23970>
EverestSchema_MyClass:insignificant-cattery;9487504950150
Inputs{MyClass}(
    a = 10
    b = 2
    c = 3
    )
60


In [21]:
mycase1 = MyClass[dict(a = 10)]

In [22]:
mycase1 = MyClass[dict(a = 10)]
mycase2 = MyClass[dict(a = 10)]
assert mycase1 is mycase2

In [23]:
assert isinstance(MyClass, Schema)
mycase = MyClass[dict(a = 10)]
assert issubclass(mycase, MyClass)
myinst = mycase()
print(myinst.foo)
assert isinstance(myinst, mycase)

60


In [24]:
oid1 = MyClass[[0, 1, 2]]
print(repr(oid1))
print(repr(oid1.hashID))

<class '__main__.MyClass[funcy.Arbitrary(0, 1, 2)]'>
'EverestSchema_MyClass[psuageuplou-skianyeew]'


In [25]:
oid2 = oid1[[3, 4, 5]]
print(repr(oid2))
print(repr(oid2.hashID))

<class '__main__.MyClass[funcy.SeqSwathe(funcy.Arbitrary(0, 1, 2), funcy.Arbitrary(3, 4, 5))]'>
'EverestSchema_MyClass[aswaeaeskou-deupspaakr]'


In [26]:
myoid = MyClass[Fn(dict(
    a = 10 ** Fn[:3],
    b = Fn[:10],
    c = 3,
    ))]
myoid

__main__.MyClass[funcy.Map(funcy.Group(a, b, c), funcy.Group(funcy.Op{opkey: pow}(10, funcy.Regular(0, 3, 1)), funcy.Regular(0, 10, 1), 3))]

In [30]:
myoid[0]().inputs

Inputs{MyClass}(
    a = 1
    b = 0
    c = 3
    )

In [32]:
myoid[10]().foo

3000

In [ ]:
myoid.incisor.sub[0]

In [ ]:
myseq = Fn(dict(
    a = 10 ** Fn[:3],
    b = Fn[:10],
    c = 3,
    ))

In [ ]:
isinstance(myseq, _generic.FuncyStrictIncisor)

In [ ]:
myinst = myoid()

In [ ]:
myinst.inputs

In [ ]:
myseq = Fn[1, 2, 3, 4, 5, 6]

In [ ]:
myseq.sub[Fn[3, 4, 5]]

In [ ]:
oid2.incisor

In [ ]:
oid2[[6, 7, 8]]

In [ ]:
oid2.incisor

In [ ]:
oid2._incisors

In [ ]:
oid1.incisor

In [ ]:
oid2.incisor

In [ ]:
issubclass(list, _generic.FuncyUnpackable)

In [ ]:
issubclass(list, _generic.FuncyBroadIncisor)